In [ ]:
GPT_CONFIG_124M={
    'vocab_size':50257,
    'context_length':1024,
    'emb_dim':768,
    'n_head':12,
    'n_layers':12,
    'drop_rate':0.1,
    'qkv_bias':False
}

In [ ]:
class GPTModel(torch.nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.tok_emb=torch.nn.Embedding(cfg['vocab_size'],cfg['emb_dim'])
        self.pos_emb=torch.nn.Embedding(cfg['context_length'],cfg['emb_dim'])
        self.drop_emb=torch.nn.Dropout(cfg['drop_rate'])

        self.trf_blocks=torch.nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg['n_layers'])])

        self.final_norm=LayerNorm(cfg['emb_dim'])#implemented
        self.output_head=torch.nn.Linear(cfg['emb_dim'], cfg['vocab_size'])

    
    def forward(self,in_idx):#input ID
        batch_size,seq_len=in_idx.shape()
        tok_emb=self.tok_emb(in_idx)
        pos_emb=self.pos_emb(torch.arange(seq_len,device=in_idx.device))
        x=tok_emb+pos_emb
        x=self.drop_emb(x)
        x=self.trf_blocks(x)#transformerblock implemented
        logits=self.output_head(x)

        return logits